<a href="https://colab.research.google.com/github/alilotfi90/Flower_Classification/blob/main/test6_ResNet50_gaussian_histogram_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter


!pip install efficientnet_pytorch
from zipfile import ZipFile
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
import torch.optim as optim
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
import torchvision.models as models




drive.mount('/content/drive')

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=8354c3ebd5c5724a96ac6b970f567ba8e666e3c639be63d4b4e4a4c00722cc9b
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch
Mounted at /content/drive


In [2]:

import zipfile


with zipfile.ZipFile('/content/drive/MyDrive/flowerimages.zip', 'r') as zip_ref:
    zip_ref.extractall('unzipped_images_flower')

In [3]:

from PIL import Image
import os

unzipped_folder = '/content/unzipped_images_flower/flowers'
output_folder = '/content/resized_images_flower'


os.makedirs(output_folder, exist_ok=True)

subdirectories = [d for d in os.listdir(unzipped_folder) if os.path.isdir(os.path.join(unzipped_folder, d))]

# seen = set()
for flower_category in subdirectories:
    input_folder = os.path.join(unzipped_folder, flower_category)
    output_category_folder = os.path.join(output_folder, flower_category)


    os.makedirs(output_category_folder, exist_ok=True)

    image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]

    for image_file in image_files:
      input_image_path = os.path.join(input_folder, image_file)
      output_image_path = os.path.join(output_category_folder, image_file)

      image = Image.open(input_image_path)

      # Ensure the image is RGB
      image = image.convert("RGB")

      # Resize the image
      target_size = (224, 224)
      image = image.resize(target_size, Image.ANTIALIAS)

      image.save(output_image_path)


from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split


max_images_per_class =60
data = []

for flower_category in subdirectories:
    category_folder = os.path.join(output_folder, flower_category)
    image_files = [f for f in os.listdir(category_folder) if f.endswith('.jpg')]
    label = subdirectories.index(flower_category)  # label = folder's name

    # Limit the number of images to max_images_per_class
    image_files = image_files[:max_images_per_class]

    for image_file in image_files:
        image_path = os.path.join(category_folder, image_file)
        data.append((image_path, label))

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)








# from PIL import Image
# import os

# unzipped_folder = '/content/unzipped_images_flower/flowers'
# output_folder = '/content/resized_images_flower'


# os.makedirs(output_folder, exist_ok=True)

# subdirectories = [d for d in os.listdir(unzipped_folder) if os.path.isdir(os.path.join(unzipped_folder, d))]

# # seen = set()
# for flower_category in subdirectories:
#     input_folder = os.path.join(unzipped_folder, flower_category)
#     output_category_folder = os.path.join(output_folder, flower_category)


#     os.makedirs(output_category_folder, exist_ok=True)

#     image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]

#     for image_file in image_files:
#       input_image_path = os.path.join(input_folder, image_file)
#       output_image_path = os.path.join(output_category_folder, image_file)

#       image = Image.open(input_image_path)

#       # Ensure the image is RGB
#       image = image.convert("RGB")

#       # Resize the image
#       target_size = (224, 224)
#       image = image.resize(target_size, Image.ANTIALIAS)

#       image.save(output_image_path)


# from torchvision import transforms
# from torch.utils.data import DataLoader, Dataset
# from sklearn.model_selection import train_test_split


# max_images_per_class =60
# data = []

# for flower_category in subdirectories:
#     category_folder = os.path.join(output_folder, flower_category)
#     image_files = [f for f in os.listdir(category_folder) if f.endswith('.jpg')]
#     label = subdirectories.index(flower_category)  # label = folder's name

#     # Limit the number of images to max_images_per_class
#     image_files = image_files[:max_images_per_class]

#     for image_file in image_files:
#         image_path = os.path.join(category_folder, image_file)
#         data.append((image_path, label))

# train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


<ipython-input-3-4b2652e337d8>:33: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = image.resize(target_size, Image.ANTIALIAS)


In [4]:
# from torch.utils.data import DataLoader, Dataset
# from sklearn.model_selection import train_test_split
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# class CustomDataset(Dataset):
#     def __init__(self, data, transform=None):
#         self.data = data
#         self.transform = transform
#         self.grayscale_count = 0  # Adding a counter for grayscale images

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         image_path, label = self.data[idx]
#         image = Image.open(image_path)

#         # we don't really need the following, we changed rare grayscale images to rgb when feeding to the colour_model
#         # Check and convert grayscale images
#         # if image.mode != 'RGB':
#         #     image = image.convert('RGB')

#         if self.transform:
#             image = self.transform(image)
#         return image, label

# class HistEqualize(object):
#     def __call__(self, img):
#         return F.equalize(img)

#     def __repr__(self):
#         return self.__class__.__name__ + '()'

# #base transform
# transform_gray = transforms.Compose([
#     transforms.Grayscale(num_output_channels=1),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485], std=[0.229])
# ])
# # Gaussian Blur Transformation
# transform_gaussian = transforms.Compose([
#     transforms.Grayscale(num_output_channels=1),
#     transforms.GaussianBlur(5),  # Kernel size of 5, you can adjust as needed
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485], std=[0.229])
# ])

# # Histogram Equalization Transformation
# transform_hist_eq = transforms.Compose([
#     transforms.Grayscale(num_output_channels=1),
#     HistEqualize(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485], std=[0.229])
# ])

# # Data Loaders for the new transformations
# train_loader_gray = DataLoader(CustomDataset(train_data, transform=transform_gray), batch_size=32, shuffle=True)
# test_loader_gray = DataLoader(CustomDataset(test_data, transform=transform_gray), batch_size=32)
# train_loader_gaussian = DataLoader(CustomDataset(train_data, transform=transform_gaussian), batch_size=32, shuffle=True)
# test_loader_gaussian = DataLoader(CustomDataset(test_data, transform=transform_gaussian), batch_size=32)
# train_loader_hist_eq = DataLoader(CustomDataset(train_data, transform=transform_hist_eq), batch_size=32, shuffle=True)
# test_loader_hist_eq = DataLoader(CustomDataset(test_data, transform=transform_hist_eq), batch_size=32)


from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import torchvision.transforms.functional as F
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        self.grayscale_count = 0  # Adding a counter for grayscale images

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]
        image = Image.open(image_path)

        # we don't really need the following, we changed rare grayscale images to rgb when feeding to the colour_model
        # Check and convert grayscale images
        # if image.mode != 'RGB':
        #     image = image.convert('RGB')

        if self.transform:
            image = self.transform(image)
        return image, label

class HistEqualize(object):
    def __call__(self, img):
        return F.equalize(img)

    def __repr__(self):
        return self.__class__.__name__ + '()'

#base transform
transform_gray = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])
])
# Gaussian Blur Transformation
transform_gaussian = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.GaussianBlur(5),  # Kernel size of 5, you can adjust as needed
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])
])

# Histogram Equalization Transformation
transform_hist_eq = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    HistEqualize(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])
])

# Data Loaders for the new transformations
train_loader_gray = DataLoader(CustomDataset(train_data, transform=transform_gray), batch_size=32, shuffle=True)
test_loader_gray = DataLoader(CustomDataset(test_data, transform=transform_gray), batch_size=32)
train_loader_gaussian = DataLoader(CustomDataset(train_data, transform=transform_gaussian), batch_size=32, shuffle=True)
test_loader_gaussian = DataLoader(CustomDataset(test_data, transform=transform_gaussian), batch_size=32)
train_loader_hist_eq = DataLoader(CustomDataset(train_data, transform=transform_hist_eq), batch_size=32, shuffle=True)
test_loader_hist_eq = DataLoader(CustomDataset(test_data, transform=transform_hist_eq), batch_size=32)

class CustomEfficientNet(nn.Module):
    def __init__(self, pretrained_model_name='efficientnet-b0', num_classes=16):
        super(CustomEfficientNet, self).__init__()
        self.embedding = nn.Conv2d(1, 3, kernel_size=1, stride=1, padding=0)
        self.eff_net = EfficientNet.from_pretrained(pretrained_model_name, num_classes=num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.eff_net(x)
        return x

from torchvision.models import resnet50

#CustomResNet50 class
class CustomResNet50(nn.Module):
    def __init__(self, num_classes=16):
        super(CustomResNet50, self).__init__()
        self.embedding = nn.Conv2d(1, 3, kernel_size=1, stride=1, padding=0)
        self.resnet = resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.resnet(x)
        return x


model_gray = CustomResNet50().to(device)
model_gray_gaussian = CustomResNet50().to(device)
model_gray_hist_eq = CustomResNet50().to(device)


# model_gray = CustomEfficientNet().to(device)  # For regular grayscale
# model_gray_gaussian = CustomEfficientNet().to(device)  # For Gaussian-blurred grayscale
# model_gray_hist_eq = CustomEfficientNet().to(device)  # For histogram-equalized grayscale


optimizer_gray = optim.SGD(model_gray.parameters(), lr=0.001, momentum=0.9)
optimizer_gray_gaussian = optim.SGD(model_gray_gaussian.parameters(), lr=0.001, momentum=0.9)
optimizer_gray_hist_eq = optim.SGD(model_gray_hist_eq.parameters(), lr=0.001, momentum=0.9)

criterion = nn.CrossEntropyLoss()

def train_test_model(model, train_loader, test_loader, optimizer, criterion, epochs_list, model_name):
    accuracies = []
    cumulative_epochs = 0
    for epochs in epochs_list:
        cumulative_epochs += epochs
        for epoch in range(epochs):
            model.train()
            running_loss = 0.0
            for i, data in enumerate(train_loader, 0):
                inputs, labels = data[0].to(device), data[1].to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                if i % 10 == 9:
                    print(f'[{cumulative_epochs + epoch + 1}, {i + 1}] loss: {running_loss / 10:.3f}')
                    running_loss = 0.0

            model.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for data in test_loader:
                    images, labels = data[0].to(device), data[1].to(device)
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            accuracy = 100 * correct / total
            accuracies.append(accuracy)
            print(f'Accuracy of {model_name} after {cumulative_epochs + epoch + 1} epochs on test images: {accuracy:.2f}%')
    return accuracies

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 158MB/s]


In [ ]:
# class CustomEfficientNet(nn.Module):
#     def __init__(self, pretrained_model_name='efficientnet-b0', num_classes=16):
#         super(CustomEfficientNet, self).__init__()
#         self.embedding = nn.Conv2d(1, 3, kernel_size=1, stride=1, padding=0)
#         self.eff_net = EfficientNet.from_pretrained(pretrained_model_name, num_classes=num_classes)

#     def forward(self, x):
#         x = self.embedding(x)
#         x = self.eff_net(x)
#         return x
# model_gray = CustomEfficientNet().to(device)  # For regular grayscale
# model_gray_gaussian = CustomEfficientNet().to(device)  # For Gaussian-blurred grayscale
# model_gray_hist_eq = CustomEfficientNet().to(device)  # For histogram-equalized grayscale


# optimizer_gray = optim.SGD(model_gray.parameters(), lr=0.001, momentum=0.9)
# optimizer_gray_gaussian = optim.SGD(model_gray_gaussian.parameters(), lr=0.001, momentum=0.9)
# optimizer_gray_hist_eq = optim.SGD(model_gray_hist_eq.parameters(), lr=0.001, momentum=0.9)

# criterion = nn.CrossEntropyLoss()

# def train_test_model(model, train_loader, test_loader, optimizer, criterion, epochs_list, model_name):
#     accuracies = []
#     cumulative_epochs = 0
#     for epochs in epochs_list:
#         cumulative_epochs += epochs
#         for epoch in range(epochs):
#             model.train()
#             running_loss = 0.0
#             for i, data in enumerate(train_loader, 0):
#                 inputs, labels = data[0].to(device), data[1].to(device)
#                 optimizer.zero_grad()
#                 outputs = model(inputs)
#                 loss = criterion(outputs, labels)
#                 loss.backward()
#                 optimizer.step()
#                 running_loss += loss.item()
#                 if i % 10 == 9:
#                     print(f'[{cumulative_epochs + epoch + 1}, {i + 1}] loss: {running_loss / 10:.3f}')
#                     running_loss = 0.0

#             model.eval()
#             correct = 0
#             total = 0
#             with torch.no_grad():
#                 for data in test_loader:
#                     images, labels = data[0].to(device), data[1].to(device)
#                     outputs = model(images)
#                     _, predicted = torch.max(outputs.data, 1)
#                     total += labels.size(0)
#                     correct += (predicted == labels).sum().item()

#             accuracy = 100 * correct / total
#             accuracies.append(accuracy)
#             print(f'Accuracy of {model_name} after {cumulative_epochs + epoch + 1} epochs on test images: {accuracy:.2f}%')
#     return accuracies

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 432MB/s]


Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0


In [5]:
# epochs_list = [10, 20,30,40,50,60]
# # Training and geting accuracies

# train_loader_gray = DataLoader(CustomDataset(train_data, transform=transform_gray), batch_size=32, shuffle=True)
# test_loader_gray = DataLoader(CustomDataset(test_data, transform=transform_gray), batch_size=32)
# train_loader_gaussian = DataLoader(CustomDataset(train_data, transform=transform_gaussian), batch_size=32, shuffle=True)
# test_loader_gaussian = DataLoader(CustomDataset(test_data, transform=transform_gaussian), batch_size=32)
# train_loader_hist_eq = DataLoader(CustomDataset(train_data, transform=transform_hist_eq), batch_size=32, shuffle=True)
# test_loader_hist_eq = DataLoader(CustomDataset(test_data, transform=transform_hist_eq), batch_size=32)

# accuracies_gray = train_test_model(model_gray, train_loader_gray, test_loader_gray, optimizer_gray, criterion, epochs_list, "Grayscale Model")
# accuracies_gaussian = train_test_model(model_gray_gaussian, train_loader_gaussian, test_loader_gaussian, optimizer_gray, criterion, epochs_list, "Gaussian Model")
# accuracies_hist_eq = train_test_model(model_gray_hist_eq, train_loader_hist_eq, test_loader_hist_eq, optimizer_gray, criterion, epochs_list, "Histogram Equalization Model")


# df = pd.DataFrame({
#     'Epochs': [sum(epochs_list[:i+1]) for i in range(len(epochs_list))],
#     'Grayscale_Model_Accuracy': accuracies_gray,
#     'Gaussian_Model_Accuracy': accuracies_gaussian,
#     'Histogram_Model_Accuracy': accuracies_hist_eq,
# })

# import matplotlib.pyplot as plt

# fig, ax =plt.subplots(figsize=(12,4))
# ax.axis('tight')
# ax.axis('off')
# the_table = ax.table(cellText=df.values,colLabels=df.columns,loc='center')
# the_table.auto_set_font_size(False)
# the_table.set_fontsize(10)
# plt.savefig("/content/drive/MyDrive/model_accuracies.jpeg")


epochs_list = [10, 20,30]
# Training and geting accuracies

train_loader_gray = DataLoader(CustomDataset(train_data, transform=transform_gray), batch_size=32, shuffle=True)
test_loader_gray = DataLoader(CustomDataset(test_data, transform=transform_gray), batch_size=32)
train_loader_gaussian = DataLoader(CustomDataset(train_data, transform=transform_gaussian), batch_size=32, shuffle=True)
test_loader_gaussian = DataLoader(CustomDataset(test_data, transform=transform_gaussian), batch_size=32)
train_loader_hist_eq = DataLoader(CustomDataset(train_data, transform=transform_hist_eq), batch_size=32, shuffle=True)
test_loader_hist_eq = DataLoader(CustomDataset(test_data, transform=transform_hist_eq), batch_size=32)

# accuracies_hist_eq = train_test_model(model_gray_hist_eq, train_loader_hist_eq, test_loader_hist_eq, optimizer_gray, criterion, epochs_list, "Histogram Equalization Model")
# accuracies_gaussian = train_test_model(model_gray_gaussian, train_loader_gaussian, test_loader_gaussian, optimizer_gray, criterion, epochs_list, "Gaussian Model")


accuracies_hist_eq = train_test_model(model_gray_hist_eq, train_loader_hist_eq, test_loader_hist_eq, optimizer_gray_hist_eq, criterion, epochs_list, "Histogram Equalization Model")
accuracies_gaussian = train_test_model(model_gray_gaussian, train_loader_gaussian, test_loader_gaussian, optimizer_gray_gaussian, criterion, epochs_list, "Gaussian Model")
accuracies_gray = train_test_model(model_gray, train_loader_gray, test_loader_gray, optimizer_gray, criterion, epochs_list, "Grayscale Model")




# df = pd.DataFrame({
#     'Epochs': [sum(epochs_list[:i+1]) for i in range(len(epochs_list))],
#     'Grayscale_Model_Accuracy': accuracies_gray,
#     'Gaussian_Model_Accuracy': accuracies_gaussian,
#     'Histogram_Model_Accuracy': accuracies_hist_eq,
# })

# import matplotlib.pyplot as plt

# fig, ax =plt.subplots(figsize=(12,4))
# ax.axis('tight')
# ax.axis('off')
# the_table = ax.table(cellText=df.values,colLabels=df.columns,loc='center')
# the_table.auto_set_font_size(False)
# the_table.set_fontsize(10)
# plt.savefig("/content/drive/MyDrive/model_accuracies.jpeg")

[11, 10] loss: 2.798
[11, 20] loss: 2.656
Accuracy of Histogram Equalization Model after 11 epochs on test images: 15.10%
[12, 10] loss: 2.310
[12, 20] loss: 2.127
Accuracy of Histogram Equalization Model after 12 epochs on test images: 44.79%
[13, 10] loss: 1.695
[13, 20] loss: 1.597
Accuracy of Histogram Equalization Model after 13 epochs on test images: 55.73%
[14, 10] loss: 1.173
[14, 20] loss: 1.121
Accuracy of Histogram Equalization Model after 14 epochs on test images: 63.54%
[15, 10] loss: 0.808
[15, 20] loss: 0.784
Accuracy of Histogram Equalization Model after 15 epochs on test images: 67.71%
[16, 10] loss: 0.583
[16, 20] loss: 0.473
Accuracy of Histogram Equalization Model after 16 epochs on test images: 71.88%
[17, 10] loss: 0.373
[17, 20] loss: 0.355
Accuracy of Histogram Equalization Model after 17 epochs on test images: 73.44%
[18, 10] loss: 0.239
[18, 20] loss: 0.248
Accuracy of Histogram Equalization Model after 18 epochs on test images: 74.48%
[19, 10] loss: 0.175
[19